# Day 1 - Lab 2: Generating a Product Requirements Document (PRD)

**Objective:** Use the structured `day1_user_stories.json` artifact from the previous lab to generate a formal, comprehensive Product Requirements Document (PRD) in markdown format.

**Estimated Time:** 60 minutes

**Introduction:**
With a validated set of user stories, we can now create a higher-level planning document: the PRD. A PRD serves as the source of truth for the product team, outlining the project's purpose, features, and requirements. In this lab, you will use an LLM to synthesize the detailed user stories into this formal document.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

This initial block sets up our environment. It adds the project's root directory to the Python path, allowing us to import our custom `utils.py` script. We then initialize the connection to our Large Language Model (LLM) and load the JSON artifact from the previous lab.

**Model Selection:**
You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in `utils.py`, such as `"gemini-2.5-flash"` or `"meta-llama/Llama-3.3-70B-Instruct"`.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the user stories JSON file and the PRD template.
- `save_artifact()`: To save our generated PRD and Pydantic model.

In [45]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, prompt_enhancer
# Initialize the LLM client. You can change the model here.
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the artifact from Lab 1
user_stories_data = load_artifact("day1_user_stories.json")
if user_stories_str:
    # Only decode if it's a string
    if isinstance(user_stories_str, str):
        try:
            user_stories_data = json.loads(user_stories_str)
        except json.JSONDecodeError as e:
            print(f" Failed to parse JSON: {e}")
            user_stories_data = []
    else:
        user_stories_data = user_stories_str
else:
    user_stories_data = []

2025-10-01 19:37:30,994 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating a Simple PRD

**Task:** Use the loaded user stories to generate a simple PRD.

**Instructions:**
1. Create a prompt that instructs the LLM to act as a Product Manager.
2. Provide the `user_stories_data` as context.
3. Ask the LLM to generate a PRD with three sections: "Introduction", "User Personas", and "Features / User Stories".

**Expected Quality:** A clean markdown document that correctly summarizes the provided user stories into the requested sections.

In [46]:
user_stories_text = json.dumps(user_stories_data, indent=2)

# TODO: Write a prompt to generate a simple PRD.
simple_prd_prompt = f"""
You are a Product Manager. Based on the following user stories, generate a Product Requirements Document (PRD) in Markdown format.

The PRD should contain the following three sections:
1. **Introduction** – a brief overview of the product and its purpose.
2. **User Personas** – a list of distinct user personas found in the stories.
3. **Features / User Stories** – list each feature as a heading followed by related user stories and acceptance criteria.
4. **User Story**
5. **Acceptance Criteria** (2–3 per story)

Use the information provided below as context:

Here are the user stories:
{user_stories_text}

Make this comprehensive and ready to use and think of all relevant items to be included.

 Based on the following project description, generate 5 detailed user stories for an onboarding productivity tool aimed at new hires and HR managers.

Project Description:
"A web platform to help companies onboard new hires efficiently. It includes checklists, document uploads, training tracking, and team introductions. It should work on desktop and mobile."

Do not include email-productivity application at all focus only on onboarding tool and resources.

"""

simple_prd_prompt = prompt_enhancer(simple_prd_prompt)

print("--- Generating Simple PRD ---")
if user_stories_data:
    simple_prd_output = get_completion(simple_prd_prompt, client, model_name, api_provider)
    print(simple_prd_output)
else:
    print("Skipping PRD generation because user stories are missing.")

2025-10-01 19:37:31,344 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Generating Simple PRD ---
# Introduction

This Product Requirements Document (PRD) outlines the specifications for a web platform designed to streamline the onboarding process for new hires in mid-to-large organizations. The platform aims to reduce time-to-productivity for new employees by 30% and automate 70% of HR's manual onboarding tasks within six months of launch. The platform will be responsive, integrating smoothly with existing HRIS systems while complying with SOC 2 Type II and GDPR standards. Agile delivery will be executed in 2-week sprints.

# User Personas

- **HR Manager**: Responsible for onboarding new hires and ensuring compliance with company policies.
- **New Hire**: Recently joined employee needing to complete onboarding tasks efficiently.
- **IT Administrator**: Manages system integrations and ensures data security compliance.
- **Team Lead**: Introduces new hires to team members and facilitates initial training sessions.

# Features & User Stories

## Feature

### Challenge 2 (Intermediate): Generating a PRD from a Template

**Task:** Instead of just listing sections, we will now provide the LLM with a formal template to ensure the PRD's structure is consistent and complete.

**Instructions:**
1. First, load the contents of `templates/prd_template.md` into a variable.
2. Create a new prompt that instructs the LLM to act as a Senior Product Manager.
3. Provide both the `user_stories_data` and the `prd_template_content` as context.
4. Instruct the LLM to populate the template with the information from the user stories, ensuring every section of the template is filled out.

> **Tip:** The template has sections like 'Success Metrics' and 'Out of Scope' that aren't in the user stories. This is your chance to guide the LLM's creativity! Instruct it to infer logical content for these sections based on the project's overall goal.

**Expected Quality:** A complete PRD that strictly follows the structure of the provided template file, demonstrating the LLM's ability to perform structured content generation.

In [47]:
# Load the PRD template
prd_template_content = load_artifact("artifacts/prd_template.md")

# Convert user stories to formatted string
user_stories_text = json.dumps(user_stories_data, indent=2)

# TODO: Write a prompt to populate the PRD template.
template_prd_prompt = f"""
You are a Senior Product Manager. Based on the user stories below, fill-in the following Product Requirements Document (PRD) template.

Please make sure that:
- All sections of the template are filled.
- Use only the provided user stories to extract information.
- For sections not explicitly covered (e.g., Success Metrics, Out of Scope), strategize content that is reasonable based on the project being an onboarding tool for new hires.

please use the following as context for information:

### User Stories:
{simple_prd_output}

### PRD Template:
{prd_template_content}

Give comprehensive with all related items so it can be ready-to-use for production. Items that are generally used and include potential costs, 
efficienty, time etc.
"""

# Generate the PRD
print("--- Generating PRD from Template ---")
if user_stories_data and prd_template_content:
    prd_from_template_output = get_completion(template_prd_prompt, client, model_name, api_provider)
    print(prd_from_template_output)
else:
    print("Skipping PRD generation because user stories or template are missing.")
    prd_from_template_output = ""


--- Generating PRD from Template ---
# Product Requirements Document: OnboardingPro

| Status | **Draft** |
| :--- | :--- |
| **Author** | Senior Product Management Team |
| **Version** | 1.0 |
| **Last Updated** | [Date] |

## 1. Executive Summary & Vision
OnboardingPro is a web platform designed to optimize the onboarding process for new hires in mid-to-large organizations. It aims to reduce time-to-productivity for new employees by 30% and automate 70% of HR's manual onboarding tasks within six months of launch. The platform will integrate seamlessly with existing HRIS systems, ensuring compliance with SOC 2 Type II and GDPR standards. Our vision is to create an efficient, engaging, and compliant onboarding experience that empowers new employees to become productive team members quickly.

## 2. The Problem

**2.1. Problem Statement:**
New hires currently face a fragmented and overwhelming onboarding experience, leading to decreased initial productivity and a high volume of repetitiv

### Challenge 3 (Advanced): Programmatic Validation with Pydantic

**Task:** We will now create a Pydantic model to represent the structure of our PRD. This allows us to programmatically validate any PRD, ensuring it meets our standards before it's accepted as a formal artifact.

**Instructions:**
1.  Prompt the LLM to generate a Pydantic model that reflects the structure of the `prd_template.md`. The model should have fields for each major section (e.g., `introduction: str`, `user_personas: List[str]`, `user_stories: List[Dict]`).
2.  Save this generated model code to a file named `app/validation_models/prd_model.py`.
3.  While we won't write the full validation script in this lab, generating the Pydantic model itself is the key advanced step. It creates a reusable, code-based standard for our documentation.

**Expected Quality:** A Python file containing a valid Pydantic model that can be used in the future to validate PRD documents automatically. This represents a shift from manual document review to automated governance.

In [48]:
# TODO: Write a prompt to generate a Pydantic model for the PRD.
# Tip: Be specific. Tell the LLM to create a class named 'ProductRequirementsDocument' and to use appropriate types from Python's 'typing' library.
pydantic_model_prompt = f"""
You are a Python engineer. Generate a Pydantic model named `ProductRequirementsDocument` to represent the structure of the following PRD template.

Each major section of the PRD should be a field in the model, using appropriate types from the `typing` module.

Use:
- `str` for textual sections like introduction, problem statement, etc.
- `List[str]` for things like user personas or goals
- `List[Dict[str, Any]]` or better-typed models for user stories and features, if needed.

Wrap your code in a single class inside a Python module. Use proper typing and include import statements.

### PRD Template:
{prd_template_content}
"""


print("--- Generating Pydantic Model for PRD ---")
if prd_template_content:
    pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider)
    
    # Clean up the code if it's wrapped in markdown fences
    if '```' in pydantic_model_code:
        pydantic_model_code = pydantic_model_code.split('```')[1].lstrip('python').strip()
    
    print("\n--- Generated Pydantic Model ---")
    print(pydantic_model_code)

    # Save the generated Pydantic model code to a file.
    model_path = "app/validation_models/prd_model.py"
    save_artifact(pydantic_model_code, model_path, overwrite=True)
else:
    print("Skipping Pydantic model generation because template is missing.")

# Finally, save the completed PRD from the intermediate challenge
if prd_from_template_output:
    save_artifact(prd_from_template_output, "day1_prd.md", overwrite=True)

--- Generating Pydantic Model for PRD ---

--- Generated Pydantic Model ---
from typing import List, Dict, Any, Optional
from pydantic import BaseModel

class ProductRequirementsDocument(BaseModel):
    status: str
    author: str
    version: str
    last_updated: str

    executive_summary_vision: str

    problem_statement: str
    user_personas_scenarios: List[str]  # List of user personas and scenarios

    goals_success_metrics: List[Dict[str, Any]]  # Each goal with KPIs and targets

    functional_requirements_user_stories: List[Dict[str, Any]]  # User stories with acceptance criteria

    non_functional_requirements: List[str]

    release_plan_milestones: List[Dict[str, str]]  # Version and target date

    out_of_scope: List[str]
    future_considerations: List[str]

    appendix_open_questions: List[Dict[str, str]]  # Open questions and dependencies

# Example instantiation

prd_example = ProductRequirementsDocument(
    status="Draft",
    author="[Your Name / Team Name]",

## Lab Conclusion

Excellent work! You have now taken the structured user stories from the first lab and synthesized them into a formal Product Requirements Document. You also created a Pydantic model to enforce the structure of this document, introducing automated governance into your workflow. The `day1_prd.md` artifact will be the primary input for Day 2, where we will begin designing our system's architecture and database.

> **Key Takeaway:** Using an LLM to populate a pre-defined template is a powerful pattern for creating consistent, high-quality documentation at scale. It combines the LLM's language skills with your required structure.